### Collecting & Analyzing Telegram Public Channel Data
Eli Lewien & Michael Baldassaro

This Jupyter Notebook contains code to enable you to: 

1. Collect messages, images, video, and audio files from public channels on the Telegram messaging app
2. Perform basic keyword searches to filter content for further manual review and analysis  

Before using this script, you will need to: 

* Create a Telegram account (if you don't have one already)
* Create an app to obtain API credentials:
    * Log in to https://my.telegram.org/
    * Click on 'API development tools'
    * Create an app to generate an api_id and api_hash 

As a first step, you will import the packages needed to execute your script and enter the API credentials you obtained when you created your app. 

Replace the values for the name, api_id, and api_hash below with your name and credentials then click execute.

This script also creates three empty folders - video, img and audio - where audio visual content contained in messages will be downloaded and stored. 

In [158]:
import os
import pandas as pd
from datetime import datetime, timezone
from telethon import functions, types
from telethon.sync import TelegramClient
import numpy as np

path_vid = 'video'; os.mkdir(path_vid) #creates empty video folder to hold videos
path_img = 'img'; os.mkdir(path_img) #image folder
path_aud = 'audio'; os.mkdir(path_aud) #audio folder

name = 'your username' #if no user name, enter 'anon' - it should work just fine!
api_id = 'your id' #the 'App api_id' from the app you created in https://my.telegram.org/apps
api_hash = 'your hash' #the 'App api_hash' from the app you created in https://my.telegram.org/apps


Next you list the Telegram handles (@userhandle) for the public channel(s) from which you want to collect messages. 

For the purposes of this example, we're using the handles of the New York Times (@nytimes) & Fox News (@FoxNewsTG)

If you wanted to collect messages from channels of US-based extremist groups, e.g., The Proud Boys (@proudboysusa), Texas Three Percenters (@TexasThreePercent], etc. or conspiracy groups, e.g., QAnon Warriors (@QAnon Warriors), American Patriot Movement (@AmericanPatriotsMovement), etc. just replace the values for channels in the code block below with the handles of channels for which you would like to collect messages then execute.

In [85]:
channels = ['nytimes', 'FoxNewsTG']

Set the time period for which you would like to collect messages. 

For this example, we set a start time of June 25, 2022 at 00:00:00 UTC and an end time of June 25, 2022 at 23:59:59 UTC to collect one full day of messages. 

Replace the parameter values for start_time and end_time with the dates and times for which you would like to collect messages then execute.  

In [86]:
start_time = datetime(2022, 6, 25, 0, 0, 0) #params: year, month, day, hour, minute, second
start_time = start_time.replace(tzinfo=timezone.utc) #sets the timezone to UTC
end_time = datetime(2022, 6, 25, 23, 59, 59)
end_time = end_time.replace(tzinfo=timezone.utc)

Now execute the following script to iterate through the channels to: 

* Collect data on the senders, messages, links, number of message views, and number of message forwards. 
* Generate a csv file of the data and store it the main folder with the date the data was collected 
* Download video, images, or audio files associated with messages and store them in folders

When you execute this script, you may be prompted to enter the phone number associated with your account and then a verification code you receive via Telegram to confirm your identity. If you do not enter your phone number and verification code, the script will run in an endless loop and no data will be collected!

Once the script has finished executing, you should see an output dataframe of the all data that was collected and stored in an object called 'df' (note: this is also what is contained in the csv file that is generated) 

Note: Depending on the volume of posts, this script may take anywhere from a few seconds to several minutes (or more) to execute. When done, the  This pulls multiple of the same post at the same timestamp but each with different view counts. Monitors have sorted posts in CSV to remove extra posts, but could clean up with script. 

In [ ]:
data = [] 
now = str(datetime.now().date())

for channel in channels:   
    async with TelegramClient(name, api_id, api_hash) as client:
        async for message in client.iter_messages('t.me/'+ str(channel)):
            if end_time > message.date and message.date > start_time:
                if message.photo: 
                    await message.download_media(file=path_img)
                elif message.video:
                    await message.download_media(file=path_vid)
                elif message.audio:
                    await message.download_media(file=path_aud)
                data.append([channel, 't.me/'+ str(channel)+ '/' + str(message.id), message.sender_id, message.date, message.forwards, message.views, message.text])
      
df = pd.DataFrame(data, columns=['channel','link', 'sender', 'date', 'forwards', 'views', 'message'])
df.to_csv('telegram' + now + '.csv', encoding='utf-8') 
df


Congratulations! In just a few lines of code, you've successfully collected message data and audio visual files from public channels of interest on Telegram.

You can now mine and analyze this data to see what content is getting the most views, generating the most forwards, and / or may be harmful in some way. 

In Myanmar, the team used a two-step approach to identify content that was harmful: 
1. Check to see if messages contained one or more keywords commonly associated with possible disinformation and hate speech in the Burmese context
2. Task staff with reviewing messages that contained such keywords to evaluate whether individual pieces of content were actually harmful. 

This approach effectively enabled the team to sift through a large volume of data and isolate potentially harmful content for manual review and analysis. 

For the purposes of this illustrative example, we're going to use an oversimplified approach to identify what proportion of messages in the New York Times and Fox News Telegram channels was related to the US Supreme Court decision to overturn Roe v. Wade, which was issued the day before the data was collected. We use a very short (and certainly non-extensive) list of keywords: 'roe' 'wade' 'supreme court', 'abortion', 'pro-choice', 'pro-life'.

To make our keyword search work:

* We create a new column in our dataframe ('text') that contains the message in all lowercase letters 
* We search for keywords found in the text and, for all that appear, add those keywords in a new column called 'keyword' 
* We assign a value of 1 if any of the keywords are found in the text - or a value of 0 where they don't - in a new column called 'mention'

Once the script has finished executing, you should see an output dataframe with additional columns 

Replace any of the values in keywords with keywords of your choice and execute the script.

In [ ]:
keywords = ['roe', 'wade', 'supreme court', 'abortion', 'pro-choice', 'pro-life']
df['text'] = pd.Series(df['message']).str.lower()

df['keyword'] = df['text'].str.findall('|'.join(keywords)).apply(set).str.join(', ')
df['mention'] = np.where(df['keyword'] == "", 0, 1)
df

(NB: alternatively, if we wanted to see which specific keywords were mentioned in a given message and assign a 1 or a 0 to signify whether that keyword appeared in the message, we could use the following script to do that).

In [ ]:
def keyword_search(message, keywords):
    if keywords in message:
        return int(1)
    else:
        return int(0)

for keyword in keywords:
    df[keyword] = df['text'].apply(lambda t: keyword_search(t, keyword))

df

While this code is pretty basic, it is a good starting point to enable you to collect data from Telegram and perform some analysis of content. There is much more that can be done using Natural Language Processing (NLP) and Social Network Analysis (SNA) methods to extract prominent keywords, evaluate sentiment, and perform topical analysis. Those approaches will be explored and documented in future notebooks.    